#### Just change the fake data here

In [1]:
#Change the data here
load('dataset_inf.Rdata')
load(url("https://github.com/zhentaoshi/Econ5821/raw/main/data_example/data_oos.Rdata"))
ls()

[1] "cpi"            "fake.testing.X" "ppi"            "real.cpi"      
[5] "real.ppi"       "real.X"         "X"

In [2]:
data=rbind(X,real.X)

#### Then just run the codes below

In [3]:
# calculate the inflation rate with two dataset
lncpi=log(cpi[13:168,2])-log(cpi[1:156,2])
lnppi=log(ppi[13:168,2])-log(ppi[1:156,2])

#### Here we use PC number 60 and lag $X_{t-12}$. Analysis see 'report_factor'.

In [4]:
cpi_ts <- ts(lncpi, start = c(1, 1), frequency = 12)
ppi_ts <- ts(lnppi, start = c(1, 1), frequency = 12)
length(cpi_ts)
data_X=cbind(data[13:198,-1],data[1:186,-1])
X_ts <- ts(data_X, start = c(1, 1), frequency = 12)
xdir=predict(prcomp(scale(X_ts)))[,1:60]
dim(xdir)

[1] 156

[1] 186  60

#### Here we use window 60, initial 60. Analysis see the files in the folder.

In [5]:
library(forecast)

fc <- function(y, h, xreg, newxreg) {
    fit <- auto.arima(y, xreg=xreg)
    forecast(fit, xreg=newxreg, h=h)
}

sq <- function(u){u^2}
lncpi_train <- cpi_ts
lnppi_train <- ppi_ts
X_pca_train <- xdir[1:156]
X_pca_test <- xdir[157:186]

e_cpi <- tsCV(lncpi_train, fc, xreg= X_pca_train, newxreg=X_pca_test, h=30, window = 60, initial = 60)
e_ppi <- tsCV(lnppi_train, fc, xreg= X_pca_train, newxreg=X_pca_test, h=30, window = 60, initial = 60)
    
pred_e_cpi=colMeans(e_cpi, na.rm=TRUE)
pred_e_ppi=colMeans(e_ppi, na.rm=TRUE)

pred_e_cpi
pred_e_ppi

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



h=1          h=2          h=3          h=4          h=5          h=6 
0.0008785963 0.0020780090 0.0032164602 0.0041593825 0.0051306529 0.0057696078 
         h=7          h=8          h=9         h=10         h=11         h=12 
0.0063839266 0.0068142109 0.0074203345 0.0082206615 0.0087832790 0.0092578807 
        h=13         h=14         h=15         h=16         h=17         h=18 
0.0096250273 0.0098899584 0.0101380985 0.0104065215 0.0105131420 0.0108692857 
        h=19         h=20         h=21         h=22         h=23         h=24 
0.0115687137 0.0122556230 0.0124568579 0.0130711778 0.0139731685 0.0146531442 
        h=25         h=26         h=27         h=28         h=29         h=30 
0.0148896297 0.0152600387 0.0147849371 0.0146247456 0.0145001306 0.0147093311

h=1         h=2         h=3         h=4         h=5         h=6 
0.004124531 0.003268161 0.005254636 0.007152289 0.009447851 0.011543312 
        h=7         h=8         h=9        h=10        h=11        h=12 
0.011583687 0.012781017 0.012321429 0.011713394 0.012050066 0.011961057 
       h=13        h=14        h=15        h=16        h=17        h=18 
0.011771753 0.013062894 0.014955761 0.016397871 0.017314250 0.018801005 
       h=19        h=20        h=21        h=22        h=23        h=24 
0.020664884 0.024436382 0.029046268 0.033423976 0.037733944 0.041979021 
       h=25        h=26        h=27        h=28        h=29        h=30 
0.046748493 0.049502328 0.052286038 0.053414921 0.058940010 0.066407955

In [6]:
#the outcome above is the predicted error for 30 period
#the predicted value is calculated as
all_cpi=rbind(cpi,real.cpi)
all_lncpi=log(all_cpi[13:198,2])-log(all_cpi[1:186,2])
all_ppi=rbind(ppi,real.ppi)
all_lnppi=log(all_ppi[13:198,2])-log(all_ppi[1:186,2])

pred_cpi=pred_e_cpi+all_lncpi[157:186,]
pred_ppi=pred_e_ppi+all_lnppi[157:186,]
cbind(pred_cpi,pred_ppi)

,pred_cpi,pred_ppi
h=1,8.710257e-03,0.1250931779
h=2,-1.249779e-02,0.1286168497
h=3,-1.036509e-02,0.1226659935
h=4,-7.241232e-03,0.1035721068
h=5,2.158307e-05,0.0913880925
h=6,2.032756e-03,0.0914280546
h=7,2.840267e-03,0.0822706130
h=8,1.104838e-02,0.0819082884
h=9,4.570910e-03,0.0780455602
h=10,5.965427e-03,0.0665084557


In [7]:
#Here we calculated the OOS-R2
all_cpi=rbind(cpi,real.cpi)
all_lncpi=log(all_cpi[13:198,2])-log(all_cpi[1:186,2])
all_ppi=rbind(ppi,real.ppi)
all_lnppi=log(all_ppi[13:198,2])-log(all_ppi[1:186,2])

tscv_rss_cpi <- var(pred_e_cpi)
tscv_tss_cpi <- var(all_lncpi[157:186,])
tscv_rsq_cpi <- 1 - (tscv_rss_cpi / tscv_tss_cpi)
tscv_rsq_cpi

tscv_rss_ppi <- var(pred_e_ppi)
tscv_tss_ppi <- var(all_lnppi[157:186,])
tscv_rsq_ppi <- 1 - (tscv_rss_ppi / tscv_tss_ppi)
tscv_rsq_ppi

[1] 0.7498492

[1] 0.9006404